<h1>Assignment 1- Task 2</h1>
This notebook covers Task 2 - Data Preparation and Analysis. We will use all of the JSON data collected during Task 1 to perform the analysis process.

In [42]:
import json
from pathlib import Path
from datetime import datetime
import pandas as pd

In [43]:
# The city locations that we would like to study
city_names = ["dublin", "ljubljana", "lyon", "luxembourg"]

Directories for data storage:

In [44]:
dir_raw = Path("raw")
# directory for storing clean pre-processed data
dir_data = Path("data")
# make sure it exists
dir_data.mkdir(parents=True, exist_ok=True)

<h1>Data Preprocessing</h1>
we get the metadata that is already stored locally by writing a read_json function. And removing unneeded data and duplicates.

In [45]:
import pandas as pd
def read_json(city_name,date):
    df = pd.read_json(city_name)
    df = df.drop(['mainStands', 'address', 'overflowStands', "shape", "lastUpdate"], axis=1)
    df["date"] = date
    df = df[["contractName", "date", "number", "name", "banking", "bonus", "connected", "overflow", "position", "status", "totalStands"]]
    return df


<h3>Data Parsing and Merging</h3>
We define a function that will parse an individual raw JSON data file.
we use merge function to integrate dataset from the same city, from different time periods.and execute the dataset to obtain daily averages

In [46]:
def parse(city_data):
    rows=[]
    row={"contractName":city_data["contractName"][0]}
    normalized_data = pd.json_normalize(city_data["totalStands"])
    xx = pd.concat([city_data[["contractName", "number","date"]], normalized_data], axis=1)
    row["date"]=xx["date"][0]
    row["capacity"]=round(xx["capacity"].mean(),2)
    row["aver_bikes"]=round(xx["availabilities.bikes"].mean(),2)
    row["aver_electricalBikes"]=round(xx["availabilities.electricalBikes"].mean(),2)
    row["aver_mechanicalBikes"]=round(xx["availabilities.mechanicalBikes"].mean(),2)
    row["aver_stands"]=round(xx["availabilities.stands"].mean(),2)
    row["sum_bikes"]=xx["availabilities.bikes"].sum()
    row["sum_stands"]=xx["availabilities.stands"].sum()
    rows.append(row)
    return pd.DataFrame(rows).set_index("contractName")

In [47]:
def merge(city):
    xx=[]
    for i in city:
        aa=parse(i)
        xx.append(aa)
    city_merge=pd.concat(xx,axis=0)
    return city_merge


<h3>loading raw datas</h3>
To load the local datas as the pandas dataframes format


In [48]:
cities = ["dublin", "ljubljana", "lyon", "luxembourg"]
dates = ["2023-11-3", "2023-11-4", "2023-11-5", "2023-11-6","2023-11-7"]
data = {}
for city in cities:
    data[city] = [read_json(f"raw/{i}/{city}", date) for i, date in enumerate(dates, 1)]
dublin_data1, ljubljana_data1, lyon_data1, luxembourg_data1 = [data[city][0] for city in cities]
dublin_data2, ljubljana_data2, lyon_data2, luxembourg_data2 = [data[city][1] for city in cities]
dublin_data3, ljubljana_data3, lyon_data3, luxembourg_data3 = [data[city][2] for city in cities]
dublin_data4, ljubljana_data4, lyon_data4, luxembourg_data4 = [data[city][3] for city in cities]
dublin_data5, ljubljana_data5, lyon_data5, luxembourg_data5 = [data[city][4] for city in cities]

dublin_data = [dublin_data1, dublin_data2, dublin_data3, dublin_data4,dublin_data5]
ljubljana_data = [ljubljana_data1, ljubljana_data2, ljubljana_data3, ljubljana_data4,ljubljana_data5]
lyon_data = [lyon_data1, lyon_data2, lyon_data3, lyon_data4,lyon_data5]
luxembourg_data = [luxembourg_data1, luxembourg_data2, luxembourg_data3, luxembourg_data4,luxembourg_data5]

In [49]:
# Merge the Dublin dataset
dublin_merge = merge(dublin_data)

# Merge the Ljubljana dataset
ljubljana_merge = merge(ljubljana_data)

# Merge the Lyon dataset
lyon_merge = merge(lyon_data)

# Merge the Luxembourg dataset
luxembourg_merge = merge(luxembourg_data)

<h3>Data Verification</h3>
check the dataset after merge

In [50]:
dublin_merge

,date,capacity,aver_bikes,aver_electricalBikes,aver_mechanicalBikes,aver_stands,sum_bikes,sum_stands
contractName,,,,,,,,
dublin,2023-11-3,31.98,11.80,4.12,7.68,20.03,1345,2283
dublin,2023-11-4,31.98,10.88,3.92,6.96,20.96,1240,2389
dublin,2023-11-5,31.98,11.43,3.96,7.47,20.43,1303,2329
dublin,2023-11-6,31.98,11.03,3.80,7.24,20.82,1257,2374
dublin,2023-11-7,31.98,10.78,3.74,7.05,21.06,1229,2401


In [51]:
dublin_data1.head(5)

,contractName,date,number,name,banking,bonus,connected,overflow,position,status,totalStands
0,dublin,2023-11-3,42,SMITHFIELD NORTH,False,False,True,False,"{'latitude': 53.349562, 'longitude': -6.278198}",OPEN,"{'availabilities': {'bikes': 24, 'electricalBi..."
1,dublin,2023-11-3,30,PARNELL SQUARE NORTH,False,False,True,False,"{'latitude': 53.3537415547453, 'longitude': -6...",OPEN,"{'availabilities': {'bikes': 0, 'electricalBik..."
2,dublin,2023-11-3,54,CLONMEL STREET,False,False,True,False,"{'latitude': 53.336021, 'longitude': -6.26298}",OPEN,"{'availabilities': {'bikes': 1, 'electricalBik..."
3,dublin,2023-11-3,108,AVONDALE ROAD,False,False,True,False,"{'latitude': 53.359405, 'longitude': -6.276142}",OPEN,"{'availabilities': {'bikes': 14, 'electricalBi..."
4,dublin,2023-11-3,20,JAMES STREET EAST,False,False,True,False,"{'latitude': 53.336597, 'longitude': -6.248109}",OPEN,"{'availabilities': {'bikes': 0, 'electricalBik..."


To check the dataset if it has missing data

In [52]:
def missing_count(data):
    missing_count = data.isna().sum()
    print("\nNumber of missing data per column:")
    print(missing_count)
missing_count(dublin_data1)


Number of missing data per column:
contractName    0
date            0
number          0
name            0
banking         0
bonus           0
connected       0
overflow        0
position        0
status          0
totalStands     0
dtype: int64


<h1>Data Analysis</h1>


<h2>City bikes Analysis</h2>
This is a bar chart comparing the average bike capacity at each bike parking point across different cities.
we can see that dublin is the most of capacity (31.98) among those cities at earch parking,and "Luxembourg is the least(19.23).

In [53]:
from pyecharts import options as opts
from pyecharts.charts import Bar

bar_data = [
    round(dublin_merge["capacity"][0], 2),
    round(ljubljana_merge["capacity"][0], 2),
    round(lyon_merge["capacity"][0], 2),
    round(luxembourg_merge["capacity"][0], 2),
]

# Create a bar chart
bar = (
    Bar()
    .add_xaxis(city_names)
    .add_yaxis("Average Capacity", bar_data)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=True),
        itemstyle_opts=opts.ItemStyleOpts(color="skyblue"),  # Set color to sky blue
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="City Average bikes Capacity"),
        xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=15)),  # Set X-axis label rotation
        yaxis_opts=opts.AxisOpts(name="Capacity"),
        legend_opts=opts.LegendOpts(selected_mode="single"),
    )
)

bar.render_notebook()



we can see that available the mechanicalBikes is nearly twice as much as using a electricalBikes at this bike platform in Dublin among this five day.

I can relate to this because Dublin often experiences rain, making it challenging for bike commuting.

There are not electricalBikes at this bike platform in Ljubljana and luxembourg.I suggest putting in a little electricalBikes in this city.

In Lyon there are nearly same figues between electricalBikes and mechanicalBikes


In [56]:
from pyecharts.charts import Page
line_data = {
    "Dublin": dublin_merge,
    "Ljubljana": ljubljana_merge,
    "Lyon": lyon_merge,
    "Luxembourg": luxembourg_merge,
}

page = Page(layout=Page.SimplePageLayout)  # Using a simple page layout

for key,data in line_data.items():
    line = (
        Line(init_opts=opts.InitOpts(width="600px", height="400px"))  
        .add_xaxis(data["date"])  
        .add_yaxis("aver_Bikes", data["aver_bikes"])
        .add_yaxis("aver_Electric Bikes", data["aver_electricalBikes"])
        .add_yaxis("aver_Mechanical Bikes", data["aver_mechanicalBikes"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title=key),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=15)), 
            yaxis_opts=opts.AxisOpts(name="Available quantity"),
        )
    )
    page.add(line)
page.render_notebook()


We can get some informations from this line.
Lyon is the largest of bikes demanded among those cities.

And On 2023-11-5, the demand is the highest compared to other days.
because It is the weekend ,Maybe a lot of family hang out and have the fun outdoor

In [57]:
page = Page(layout=Page.SimplePageLayout)  # Using a simple page layout

for key,data in line_data.items():
    line = (
        Line(init_opts=opts.InitOpts(width="600px", height="400px"))  
        .add_xaxis(data["date"])  
        .add_yaxis("bike parking space", data["sum_stands"])
        .add_yaxis("Available bikes", data["sum_bikes"])
        .set_global_opts(
            title_opts=opts.TitleOpts(title=key),
            xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=15)), 
            yaxis_opts=opts.AxisOpts(name="Total Quantity"),
        )
    )
    page.add(line)
page.render_notebook()


<h1>Contract Analysis</h1>

Lyon is the most of contract count among those cities(427)

Luxembourg is the least only (84)

In Lyon, there is a high demand for bicycles, so many stations have been set up

In [58]:
from pyecharts import options as opts
from pyecharts.charts import Line

# Sample data
city_data = {
    "Dublin": dublin_data1,
    "Ljubljana": ljubljana_data1,
    "Lyon": lyon_data1,
    "Luxembourg": luxembourg_data1
}

line_data = []
for city_name, city_df in city_data.items():
    contract_count = len(city_df)
    line_data.append(contract_count)

line = (
    Line()
    .add_xaxis(city_names)  
    .add_yaxis("Contract Count", line_data, markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")])) 
    .set_global_opts(
        title_opts=opts.TitleOpts(title="City Contract Count "),
        xaxis_opts=opts.AxisOpts(type_="category", axislabel_opts=opts.LabelOpts(rotate=15)), 
        yaxis_opts=opts.AxisOpts(name="Count"),
    )
)

line.render_notebook()


Lyon has around 50% of bike stations that accept card payments, 

while Dublin and Luxembourg have less than 10%, and Ljubljana has no stations that accept card payments

In [59]:
true_ratios = []


for city_name, city_df in city_data.items():
    banking_counts = city_df["banking"].value_counts(normalize=True)
    true_ratio = banking_counts.get(True, 0) * 100
    true_ratios.append(true_ratio)

bar = (
    Bar()
    .add_xaxis(city_names)
    .add_yaxis("Proportion (%)", true_ratios)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Banking True Proportion in Four Cities"),
        xaxis_opts=opts.AxisOpts(name="City"),
        yaxis_opts=opts.AxisOpts(name="Proportion (%)", max_=70),
    )
)

bar.render_notebook()

Is there cashback?

It seems like that lyon is only one city having cashback and just in a few parking point

In Lyon, there might be too many stations, exceeding expectations, which has resulted in some areas having no usage at all. Hence, cashback is offered to attract customers.

In [60]:
import pandas as pd
from pyecharts.charts import Bar
from pyecharts import options as opts

false_ratios = []

for city_name, city_df in city_data.items():
    bonus_counts = city_df["bonus"].value_counts(normalize=True)
    false_ratio = bonus_counts.get(True, 0) * 100
    false_ratios.append(false_ratio)

bar = (
    Bar()
    .add_xaxis(city_names)
    .add_yaxis("Proportion (%)", false_ratios)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Bonus True Proportion in Four Cities"),
        xaxis_opts=opts.AxisOpts(name="City"),
        yaxis_opts=opts.AxisOpts(name="Proportion (%)", max_=40),
    )
)

bar.render_notebook()

In [61]:
normalized_data = pd.json_normalize(dublin_data1["totalStands"])
dublin_Stands = pd.concat([dublin_data1[["contractName", "name"]], normalized_data], axis=1)
normalized_data = pd.json_normalize(ljubljana_data1["totalStands"])
ljubljana_Stands = pd.concat([ljubljana_data1[["contractName", "name"]], normalized_data], axis=1)
normalized_data = pd.json_normalize(lyon_data1["totalStands"])
lyon_Stands = pd.concat([lyon_data1[["contractName", "name"]], normalized_data], axis=1)
normalized_data = pd.json_normalize(luxembourg_data1["totalStands"])
luxembourg_Stands = pd.concat([luxembourg_data1[["contractName", "name"]], normalized_data], axis=1)

A box plot records the stands on November 3, 2023, allowing us to visually see the maximum, minimum, 
and median values of remaining stands at different bike stations in different areas.

In [62]:
from pyecharts.charts import Boxplot
from pyecharts import options as opts

boxplot_data = []
city_data = {
    "Dublin": dublin_Stands,
    "Ljubljana": ljubljana_Stands,
    "Lyon": lyon_Stands,
    "Luxembourg": luxembourg_Stands
}
for city_name, city_df in city_data.items():
    aa=city_df["availabilities.stands"].tolist()
    boxplot_data.append(aa)
boxplot = Boxplot()
boxplot.add_xaxis(city_names)
boxplot.add_yaxis("stands boxplot", boxplot.prepare_data(boxplot_data))
boxplot.set_global_opts(title_opts=opts.TitleOpts(title="2023-11-3 stands boxplot"))
boxplot.render_notebook()

<h1>IN CONCLUSION</h1>

Closing some unnecessary bike stations in Lyon, adding more mechanical bikes in Ljubljana and Luxembourg, and increasing the number of stations in Dublin makes sense because the results indicate very high demand